In [1]:
import numpy as np
import cvxpy as cp
from scipy.io import loadmat

Load $Y_{bus}$ matrix from MATLAB

In [2]:
# Note: in per-unit
Y = loadmat('Ybus')['Ybus'].todense()

Problem parameters

In [3]:
# Number of buses
N = Y.shape[0]

# Susceptance matrix. Recall in DC power flow, G = 0 and shunt admittances are zero.
B = np.array(np.imag(Y))
B = B - np.diag(np.diag(B))
B = B - np.diag(np.sum(B, axis=0))

# System base, in MVA
base = 100

# The load consumed at each bus
p_d = np.array([0, 40, 150, 80, 130])/base

# Matrix defining the cost of the generators
# The i,j element is the coefficient of the power produced at the ith generator raised to the j-1 power
# Copied from PowerWorld
C = np.array(
    [
        [373.5, 10, 0.016],
        [403.6, 8, 0.018],
        [253.2, 12, 0.018]
    ]
)

# Minimum and maximum generator outputs, taken from PowerWorld
p_min = np.array([100, 150, 0])/base
p_max = np.array([400, 500, 300])/base

# The buses with generators and loads
G = [1, 2, 4]
L = [2, 3, 4, 5]

First, we'll solve the normal supply-side OPF. Re-index the problem such that the generator buses precede the non-generator buses. Within each block, keep the order the same.

In [4]:
NG = [n for n in range(1,N+1) if n not in G]
idx = np.array(G+NG)-1
B = B[idx][:,idx]
p_d = p_d[idx]

Solve DC OPF. Line flow limits not yet implmented, but they are not binding for this example. Confirm that the total cost is consistent with PowerWorld.

In [5]:
p_g = cp.Variable(len(G))
delta = cp.Variable(N)
constraints = [
    p_g >= p_min,
    p_g <= p_max,
    cp.hstack([p_g,np.zeros(len(NG))])-p_d == -B@delta
]
cp.Problem(
    cp.Minimize(cp.vec(C.T)@cp.vec(cp.vstack([(base*p_g)**n for n in range(3)]))),
    constraints
).solve()

5840.788888888889

In [6]:
constraints[2].dual_value/100

array([-14.608, -14.608, -14.608, -14.608, -14.608])

Now let's solve a demand-side problem with zero marginal cost, fixed supply, and elastic demand. For this minimal example, we make the follwing assumptions:
- Each load submits a demand curve. The system operator accepts the maximum-price bid until the renewable output is met.
- Each load can consume as much power as desired.
- For each load, elasticity is constant. That is, the marginal benefit curve for load $i$ is given by $MB_i(P)=K_iP^{\frac{1}{e_i}}$, where $e_i<-1$ is the elasticity. Note that this formulation insures that marginal benefit is always positive.
- Each load $i$ has a baseline demand $\underline{P}_i$ which it must always consume. Total benefit is measured in excess of the baseline.

For simplicity, suppose each generator supplies its optimal output from the supply-side problem.

In [7]:
p_g = np.hstack([p_g.value, np.zeros(len(NG))])

Again, reindex the system, this time with the load buses preceding the non-load buses

In [8]:
B = B[idx][:,idx]
p_d = p_d[idx]
NL = [n for n in range(1,N+1) if n not in L]
idx = np.array(L+NL)-1
B = B[idx][:,idx]
p_g = p_g[idx]
p_d = p_d[idx]

For simplicity, we choose $e_1=\dots=e_L=-0.2$. Furthermore, choose $K_i$ by taking the marginal benefit vector to be $MB(P_d^*)=\lambda\mathbf{1}$ where $P_d^*$ is the load vector given by PowerWorld and $\lambda$ is set to $14.62.

In [9]:
e = -0.2
price = 14.620
K = price/(p_d[:len(L)])**(1/e)
p_lower = 0.45*p_d[:len(L)]

Solve! To avoid numerical errors, the objective function is in per-unit, so it should not be compared to the supply-side case.

In [10]:
p_d = cp.Variable(len(L))
delta = cp.Variable(N)
constraints = [
    p_d >= p_lower,
    p_g-cp.hstack([p_d,np.zeros(len(NL))]) == -B@delta,
]
cp.Problem(
    cp.Maximize(e/(1+e)*cp.sum(cp.multiply(K,(p_d)**(1/e+1)-(p_lower)**(1/e+1)))),
    constraints
).solve()

341.9110165792025

The load dispatch and the LMPs are similar to the supply-side problem. In fact, the optimal consumptions are equal to the fixed loads from the supply-side problem (not sure why).

In [11]:
p_d.value*base

array([ 39.99924691, 149.99949232,  79.99975268, 130.00150794])

In [12]:
constraints[1].dual_value

array([-14.62071111, -14.62071111, -14.62071111, -14.62071111,
       -14.62071111])